# Import Estimator(PyTorch) API Module

In [1]:
import sys
sys.path.append('../../')

In [2]:
from sdrdev.aistudio.torchestimator import TorchEstimator

In [3]:
# from sdr.aistudio.torchestimator import TorchEstimator

In [4]:
print(TorchEstimator)

<class 'sdrdev.aistudio.torchestimator.TorchEstimator'>


<div class="alert alert-block alert-info">
<h1>Example(MNIST-torchvision)</h1>
</div>

<div class="alert alert-block alert-warning">
    <p>Customize Neural Network Below</p>
    <p>Do not change class name <b>'Net'</b></p>
</div>

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

<div class="alert alert-block alert-info">
<p>Feel free to change dataset name</p>
    <p>Do not change class name <b>'DatasetLoader'</b> and function names</p>
</div>

In [ ]:
from torchvision import datasets, transforms
import os

class DatasetLoader:    
    def __init__(self,dataset_path="./"):
        ########## WRITE DATASET LOADER CODE HERE ##########
        data_dir = os.path.join(dataset_path,"Torchvision_MNIST")
        mnist_transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))])
        train_dataset = datasets.MNIST(data_dir, transform=mnist_transform, train=True,  download=True)        
        test_dataset  = datasets.MNIST(data_dir, transform=mnist_transform, train=False, download=True)
        
        ### train_ds and valid_ds MUST BE TensorDataset(or ImageFolder or Torch Dataset Format)
        self.train_dataset = train_dataset
        self.valid_dataset = test_dataset
        self.test_dataset = test_dataset
        ####################################################
    
    def get_train_dataset(self, validation=True):        
        if validation is True:
            return self.train_dataset, self.valid_dataset
        else:
            return self.train_dataset
    
    def get_test_dataset(self):
        return self.test_dataset

In [ ]:
script_params = {
    'problem-type':'classification',
    'epochs':2,
    'batch-size':64,
    'test-batch-size':128,
    'lr':0.01,
    'momentum':0.5,
    'seed':42,
    'log-interval':10,
    'nprocs':8,    
    'loss':'nll_loss', # if output layer uses softmax, use this loss function
    #'loss':'cross_entropy', # if not above, use this loss function
    'optimizer':'SGD',
    'validation': True,
}

In [ ]:
estimator = TorchEstimator(script_params=script_params)

In [ ]:
estimator.fit(job_title="Torchvision-MNIST-CNN(4 GPU)")

In [ ]:
estimator.status()

In [ ]:
estimator.monitor(timeout=60.0)

### if you want to cancel the job, execute this line.

In [ ]:
estimator.cancel()

### predict by the model (After job has been finished.)

In [ ]:
estimator.predict()

In [ ]:
estimator.report()

### register model

In [ ]:
estimator.register_model(model_name="mnist-tutorial-model2")

# Prediction by cpu with user custom code

### get model

It works when the training part of the code above is completed and the kernel is kept active.

In [ ]:
model = estimator.get_model()

### custom code

In [ ]:
from torchvision import datasets, transforms
import os
import torch

data_dir = "./dataset/MNIST-TEST"
mnist_transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))])
test_dataset  = datasets.MNIST(data_dir, transform=mnist_transform, train=False, download=True)
batch_size = 32
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

classes = ('0','1','2','3','4','5','6','7','8','9')
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
with torch.no_grad():
    for idx, data in enumerate(testloader):
        images, labels = data        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)        
        c = (predicted == labels).squeeze()
        for i in range(len(data)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %2s : %3d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))